In [10]:
!pip install pandas
!pip install seaborn
!pip install plotly.express
!pip install folium
!pip install sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import folium
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV


[notice] A new release of pip available: 22.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip


In [11]:
url='https://raw.githubusercontent.com/4GeeksAcademy/random-forest-project-tutorial/main/titanic_train.csv'
df=pd.read_csv(url, sep=',')
df.shape

(891, 12)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Como existe un desabalance me tengo que fijar mas en las medidas de f1 y no tanto en accuracy

In [13]:
df['Survived'].value_counts()/len(df['Survived'])

0    0.616162
1    0.383838
Name: Survived, dtype: float64

Decision tree, logistic regression, entre otros. 
Algoritmo mas complejos se necesitan mas datos. 
Criterio para analizar random forest es mas complejo para interpretar, deberiamos de analizar cada arbol y realizar el promedio.


In [14]:
df=df.drop(columns=['Name'])

In [15]:
df['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [16]:
df['Sex']=df['Sex'].map({'male':1,'female':0})
df['Sex']

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex, Length: 891, dtype: int64

In [17]:
df['Ticket'].nunique()

681

In [18]:
df=df.drop(columns=['Ticket'])

In [19]:
df['Cabin'].nunique()
df=df.drop(columns=['Cabin'])

In [20]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [21]:
df['Embarked']=df['Embarked'].map({'S':2,'C':1,'Q':0})
df['Embarked']

0      2.0
1      1.0
2      2.0
3      2.0
4      2.0
      ... 
886    2.0
887    2.0
888    2.0
889    1.0
890    0.0
Name: Embarked, Length: 891, dtype: float64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     889 non-null    float64
dtypes: float64(3), int64(6)
memory usage: 62.8 KB


In [23]:
df['Age'][np.isnan(df['Age'])]=df['Age'].mean()


/tmp/ipykernel_2044/211817231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'][np.isnan(df['Age'])]=df['Age'].mean()


In [24]:
df['Embarked'].value_counts()

2.0    644
1.0    168
0.0     77
Name: Embarked, dtype: int64

In [25]:
df['Embarked'][np.isnan(df['Embarked'])]=2

/tmp/ipykernel_2044/960493807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Embarked'][np.isnan(df['Embarked'])]=2


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int64  
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     891 non-null    float64
dtypes: float64(3), int64(6)
memory usage: 62.8 KB


In [27]:
X=df.drop(columns=['Survived'])
y=df['Survived']
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=520, test_size=0.2)

In [28]:
modelo=RandomForestClassifier(n_estimators=50,random_state=145)

In [29]:
modelo.fit(X_train,y_train)

RandomForestClassifier(n_estimators=50, random_state=145)

In [30]:
y_train_pred=modelo.predict(X_train)
y_test_pred=modelo.predict(X_test)

In [31]:
y.value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [32]:
train_report=metrics.classification_report(y_pred=y_train_pred,y_true=y_train)
print(train_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       456
           1       1.00      1.00      1.00       256

    accuracy                           1.00       712
   macro avg       1.00      1.00      1.00       712
weighted avg       1.00      1.00      1.00       712



In [33]:
test_report=metrics.classification_report(y_pred=y_test_pred,y_true=y_test)
print(test_report)

              precision    recall  f1-score   support

           0       0.75      0.95      0.83        93
           1       0.92      0.65      0.76        86

    accuracy                           0.80       179
   macro avg       0.83      0.80      0.80       179
weighted avg       0.83      0.80      0.80       179



f1 score del train es 1 y del test es 0.76, el algoritmo esta memorizando la data. 
estaria mas conforme con un train .78 y test.76

In [34]:
modelo=RandomForestClassifier(n_estimators=5,random_state=42,max_depth=5)

In [35]:
modelo.fit(X_train,y_train)

RandomForestClassifier(max_depth=5, n_estimators=5, random_state=42)

In [36]:

y_train_pred=modelo.predict(X_train)
y_test_pred=modelo.predict(X_test)

In [37]:
train_report=metrics.classification_report(y_pred=y_train_pred,y_true=y_train)
print(train_report)

              precision    recall  f1-score   support

           0       0.84      0.97      0.90       456
           1       0.92      0.67      0.77       256

    accuracy                           0.86       712
   macro avg       0.88      0.82      0.84       712
weighted avg       0.87      0.86      0.85       712



In [38]:
test_report=metrics.classification_report(y_pred=y_test_pred,y_true=y_test)
print(test_report)

              precision    recall  f1-score   support

           0       0.69      0.95      0.80        93
           1       0.90      0.53      0.67        86

    accuracy                           0.75       179
   macro avg       0.79      0.74      0.73       179
weighted avg       0.79      0.75      0.74       179



In [39]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
#Criterio
criterion=['gini','entropy']
# Create the random grid
random_grid = {'n_estimators': n_estimators,
#'max_features': max_features, # Son muy pocas variables por lo cual no vale la pena aplicarlo
'max_depth': max_depth,
'min_samples_split': min_samples_split,
'min_samples_leaf': min_samples_leaf,
'bootstrap': bootstrap,
'criterion':criterion}
print(random_grid)

{'n_estimators': [200, 233, 266, 300, 333, 366, 400, 433, 466, 500], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False], 'criterion': ['gini', 'entropy']}


In [40]:
classif_grid=RandomForestClassifier(random_state=1107)
classif_grid_random=RandomizedSearchCV(estimator=classif_grid,n_iter=100,cv=5,random_state=1107,param_distributions=random_grid)

In [41]:
classif_grid_random.fit(X_train,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1107),
                   n_iter=100,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 233, 266, 300,
                                                         333, 366, 400, 433,
                                                         466, 500]},
                   random_state=1107)

In [42]:
classif_grid_random.best_params_

{'n_estimators': 333,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_depth': 20,
 'criterion': 'gini',
 'bootstrap': True}

In [43]:
modelo_final=RandomForestClassifier(n_estimators= 333, min_samples_split= 10,min_samples_leaf=1,max_depth= 20,criterion='gini',
bootstrap= True)

In [44]:
modelo_final.fit(X_train, y_train)
y_train_final=modelo_final.predict(X_train)
y_test_final=modelo_final.predict(X_test)

In [45]:
train_report_final=metrics.classification_report(y_pred=y_train_final,y_true=y_train)
print(train_report_final)

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       456
           1       0.95      0.84      0.89       256

    accuracy                           0.92       712
   macro avg       0.93      0.90      0.92       712
weighted avg       0.93      0.92      0.92       712



In [46]:
train_report_final=metrics.classification_report(y_pred=y_test_final,y_true=y_test)
print(train_report_final)

              precision    recall  f1-score   support

           0       0.73      0.95      0.82        93
           1       0.91      0.62      0.74        86

    accuracy                           0.79       179
   macro avg       0.82      0.78      0.78       179
weighted avg       0.82      0.79      0.78       179



In [48]:
import pickle
pickle.dump(modelo_final, open('../models/modelo_final.pickle', 'wb'))

In [49]:
modelo = pickle.load(open('../models/modelo_final.pickle', 'rb')) # lo leemos
modelo.predict(X_test) # lo usamos para predecir nueva X_test

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0])